# Fundamentals of Software Systems (FSS)
**Software Evolution – Part 02 Assignment**

## Submission Guidelines

To correctly complete this assignment you must:

* Carry out the assignment in a team of 2 to 4 students.
* Carry out the assignment with your team only. You are allowed to discuss solutions with other teams, but each team should come up its own personal solution. A strict plagiarism policy is going to be applied to all the artifacts submitted for evaluation.
* As your submission, upload the filled Jupyter Notebook (including outputs) together with the d3 visualization web pages (i.e. upload everything you downloaded including the filled Jupyter Notebook plus your `output.json`)
* The files must be uploaded to OLAT as a single ZIP (`.zip`) file by 2023-12-04 18:00.


## Group Members
* Adam Bauer, 20-744-964
* Alessio Brazeol TODO
* Paul TODO

## Task Context

In this assigment we will be analyzing the _elasticsearch_ project. All following tasks should be done with the subset of commits from tag `v1.6.0` to tag `v2.0.0`.

##  Preprocessing

In [ ]:
%pip install plotly
%pip install tdqm
%pip install nbformat

In [11]:
import os
from pydriller import Repository, Git
from tqdm import tqdm
from collections import defaultdict, Counter
import plotly.express as px
import matplotlib.pyplot as plt
import subprocess
import platform
import json

In [2]:
# clone the project -> easier setup (will not clone if folder already exists or is not empty)
url = "https://github.com/elastic/elasticsearch"
repo_path = os.path.join(os.getcwd(), 'elasticsearch')
clone = f"git clone {url} {repo_path}" 

os.system(clone)

128

In [3]:
# define Repository
gr = Git(repo_path)
from_tag = "v1.6.0"
to_tag = "v2.0.0"
since_time = gr.get_commit_from_tag(from_tag).committer_date
to_time = gr.get_commit_from_tag(to_tag).committer_date

repo = Repository(repo_path, since=since_time, to=to_time)
print(f"Analyzing repo from {from_tag} to {to_tag}, which are commits from {since_time} to {to_time}")


Analyzing repo from v1.6.0 to v2.0.0, which are commits from 2015-06-09 13:35:08+00:00 to 2015-10-21 23:01:03+02:00


In [ ]:
from enum import Enum 

class Modification(Enum):
    ADDED = "Lines added"
    REMOVED = "Lines removed"
    TOTAL = "Lines added + lines removed"
    DIFF = "Lines added - lines removed"

## Task 1: Author analysis

In the following, please consider only `java` files.

The first task is to get an overview of the author ownership of the _elasticsearch_ project. In particular, we want to understand who are the main authors in the system between the two considered tags, the authors distribution among files and the files distribution among authors. To this aim, perform the following:

* create a dictionary (or a list of tuples) with the pairs author => number of modified files
* create a dictionary (or a list of tuples) with the pairs file => number of authors who modified the file
* visualize the distribution of authors among files: the visualization should have on the x axis the number of authors per file (from 1 to max), and on the y axis the number of files with the given number of authors (so for example the first bar represent the number of files with single author)
* visualize the distribution of files among authors: the visualization should have on the x axis the number of files per author (from 1 to max), and on the y axis the number of authors that own the given number of files (so for example the first bar represent the minor contributors, i.e., the number of authors who own 1 file)

Comment the two distribution visualizations.



Now, let's look at the following 3 packages in more details:

1. `src/main/java/org/elasticsearch/common`
2. `src/main/java/org/elasticsearch/rest`
3. `src/main/java/org/elasticsearch/cluster`

Create a function that, given the path of a package and a modification type (see class Modification below), returns a dictionary of authors => number, where the number counts the total lines added or removed or added+removed or added-removed (depending on the given Modification parameter), for the given package. To compute the value at the package level, you should aggregate the data per file.

Using the function defined above, visualize the author contributions (lines added + lines removed). The visualization should have the author on the x axis, and the total lines on the y axis. Sort the visualization in decreasing amount of contributions, i.e., the main author should be the first.

Compare the visualization for the 3 packages and comment.

### Part 1

In [ ]:
def create_dictionary(repo):
    authors_dict = defaultdict(set)
    files_dict = defaultdict(set)

    commits = [commit for commit in repo.traverse_commits()]

    for commit in tqdm(commits, desc="Analyzing commits", unit="commit"):
        for file in commit.modified_files:
           filename = file.new_path
           
           authors_dict[commit.author.name].add(filename) #since dict is a set, we don't have to care about duplicities
           files_dict[filename].add(commit.author.name) #since dict is a set, we don't have to care about duplicities
           
    # perform counting on both dictionaries to get the numbers
    authors_dict = {author: len(files) for author, files in authors_dict.items()}
    files_dict = {file: len(authors) for file, authors in files_dict.items()}
    return authors_dict, files_dict

In [ ]:
# visualuse them
authors_dict, files_dict = create_dictionary(repo)


In [ ]:
def plot_bar_chart(dictionary, title, ylabel, xlabel, top_n=10):
    # Counting the occurrences in the dictionary
    counts = Counter(dictionary.values())

    # Getting the top n most common elements
    top_counts = dict(counts.most_common())

    # Preparing data for plotting
    values = list(top_counts.keys())
    frequencies = list(top_counts.values())

    # Plotting
    plt.figure(figsize=(50, 5))
    plt.bar( frequencies, values)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)

    # Adjusting x-ticks to show only labels for used values
    plt.xticks([val for val in values if val in dictionary.values()])

    plt.xticks(rotation=90)
    plt.show()


def plot_interactive_bar_chart(dictionary, title, ylabel, xlabel, top_n=10):
    # Counting the occurrences in the dictionary
    counts = Counter(dictionary.values())

    # Sorting the counts dictionary by values in descending order
    sorted_counts = dict(sorted(counts.items(), key=lambda item: item[0]))

    # Getting the top n most common elements after sorting

    # Preparing data for plotting
    values = list(sorted_counts.keys())
    frequencies = list(sorted_counts.values())

    # Creating an interactive bar chart
    fig = px.bar(x=values, y=frequencies, labels={'x': xlabel, 'y': ylabel}, title=title)
    fig.update_layout(xaxis={'type':'category'})
    fig.show()



plot_interactive_bar_chart(files_dict, "How many files are used by how many authors (so for example the first bar represent the number of files with single author)", "Number of authors", "Number of files")
plot_interactive_bar_chart(authors_dict, "How many authors have interacted with how many files (so for example the first bar represent the minor contributors, i.e., the number of authors who own 1 file)", "Number of authors", "Number of files")

### TODO add comment for the visualisations

### Part 2

In [ ]:
def get_modified_value(file, modification_type):
    if(modification_type not in Modification):
        raise ValueError("Invalid modification type")
    
    if(modification_type == Modification.ADDED):
        return file.added_lines
    elif(modification_type == Modification.REMOVED):
        return file.deleted_lines
    elif(modification_type == Modification.TOTAL):
        return file.added_lines + file.deleted_lines
    elif(modification_type == Modification.DIFF):
        return file.added_lines - file.deleted_lines

def get_authors_numbers(package_paths, modification_types):
    if(not isinstance(package_paths, list)):
        raise ValueError("Package paths must be a list")
    if(not isinstance(modification_types, list)):
        raise ValueError("Modification types must be a list")
    
    # restrict repo to the previous defined tags
    repo = Repository(repo_path, since=since_time, to=to_time)
    commits = [commit for commit in repo.traverse_commits()]

    authors_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0)))
    for commit in tqdm(commits, desc="Analyzing commits", unit="commit"):
        for file in commit.modified_files:
            filename = file.new_path if file.new_path else file.old_path

            for package_path in package_paths:
                if filename.startswith(package_path):
                    for modification_type in modification_types:
                        authors_dict[package_path][modification_type][commit.author.name] += get_modified_value(file, modification_type)

    return authors_dict   

In [ ]:
# Be aware, they have moved from core into server folder in 2018
packages = ["core\\src\\main\\java\\org\\elasticsearch\\common", "core\\src\\main\\java\\org\\elasticsearch\\rest", "core\\src\\main\\java\\org\\elasticsearch\\cluster"]
modifications = [Modification.ADDED, Modification.REMOVED, Modification.TOTAL, Modification.DIFF]
analysis_packages = get_authors_numbers(packages, modifications)

In [ ]:
# def graph(data):
#     # sort array Author: number of lines
#     data = dict(sorted(data.items(), key=lambda item: item[1]))
#     # split into 2 arrays
#     authors = data.keys()
#     lines = data.values()
#     # create bar chart
#     plt.figure(figsize=(150, 5))
#     plt.bar(authors, lines)
#     plt.xticks(rotation=90)
#     plt.show()

import plotly.express as px
def graph_interactive(data, title):
    # Sort the dictionary by values (number of lines)
    sorted_data = dict(sorted(data.items(), key=lambda item: item[1]))

    # Splitting the sorted dictionary into authors and lines
    authors = list(sorted_data.keys())
    lines = list(sorted_data.values())

    # Creating an interactive bar chart using Plotly
    fig = px.bar(x=authors, y=lines)
    fig.update_layout(
        title=title,
        xaxis_title="Authors",
        yaxis_title="Number of Lines",
        xaxis={'categoryorder':'total descending'}
    )
    fig.show()


for pack in packages:
    graph_interactive(analysis_packages[pack][Modification.TOTAL], "Package: "+pack)


### Todo comment on the analysis

## Task 2: Knowledge loss

We now want to analyze the knowledge loss when the main contributor of the analyzed project would leave. For this we will use the circle packaging layout introduced in the "Code as a Crime Scene" book. This assignment includes the necessary `knowledge_loss.html` file as well as the `d3` folder for all dependencies. Your task is to create the `output.json` file according to the specification below. This file can then be visualized with the files provided.

For showing the visualization, once you have the output as `output.json` you should

* make sure to have the `knowledge_loss.html` file in the same folder
* start a local HTTP server in the same folder (e.g. with python `python3 -m http.server`) to serve the html file (necessary for d3 to work)
* open the served `knowledge_loss.html` and look at the visualization

Based on the visualization, comment on how is the project in terms of project loss and what could happen if the main contributor would leave.


### Output Format for Visualization

* `root` is always the root of the tree
* `size` should be the total number of lines of contribution
* `weight` can be set to the same as `size`
* `ownership` should be set to the percentage of contributions from the main author (e.g. 0.98 for 98% if contributions coming from the main author)

```
{
  "name": "root",
  "children": [
    {
      "name": "test",
      "children": [
        {
          "name": "benchmarking",
          "children": [
            {
              "author_color": "red",
              "size": "4005",
              "name": "t6726-patmat-analysis.scala",
              "weight": 1.0,
              "ownership": 0.9,
              "children": []
            },
            {
              "author_color": "red",
              "size": "55",
              "name": "TreeSetIterator.scala",
              "weight": 0.88,
              "ownership": 0.9,
              "children": []
            }
          ]
        }
      ]
    }
  ]
}

{
  "name": "root",
  "children": [
    {
      "name": ".settings",
      "children": [
        {
              "name": "org.eclipse.core.resources.prefs",
              "val1": 0,
              "val2": 6,
              "children": []
        },
        {
              "name": "org.eclipse.jdt.core.prefs",
              "val1": 0,
              "val2": 18,
              "children": []
        },
      
          ]
        },
        {
      "name": "core",
      "children": [
          {
                "name": "license.txt",
                "val1": 0,
                "val2": 6,
                "children": []
          },
          {
                "name": "lsrc",
                "children": [
                  {
                        "name": "core.prefs",
                        "val1": 0,
                        "val2": 18,
                        "children": []
                  },
                ]
          },
      
          ]
        }
      ]
    }
  ]
}
```

### JSON Export

For exporting the data to JSON you can use the following snippet:

```
import json

with open("output.json", "w") as file:
    json.dump(tree, file, indent=4)
```

In [12]:
# Determine the main contributor 0> look at github stats:) Shay Banon (kimchy)
# UNIX was not tested!!!!
import shlex
def count_lines_total_and_by_authors(file_path, author_names):
    if(os.getcwd() != repo_path):
        raise ValueError("You must be in the repository folder to run this function")
    
    os_type = platform.system()
    # Create a grep pattern that matches any of the authors
    

    # needs system path 
    # Count total lines in the file
    if os_type == 'Windows':
        total_lines_cmd = f"powershell -Command \"(Get-Content {file_path} | Measure-Object).Count\""
    else:
        total_lines_cmd = f"wc -l < {file_path}"

    # print()
    total_lines_result = subprocess.run(total_lines_cmd, shell=True, capture_output=True, text=True)
    total_lines = int(total_lines_result.stdout.strip())

    # remove repo path from file path  (fix the slash that is missint at the end of repopath, otherwise reative)
    # FIX

   
    relative_file_path = file_path.replace((repo_path+"\\"), '')
    # Count lines by authors

    # needs relative path
    if os_type == 'Windows':

        patterns = ', '.join([f"'author {author}'" for author in author_names])
        author_lines_cmd = f"(git blame --line-porcelain {relative_file_path} | Select-String -Pattern {patterns}).Count"
        author_lines_cmd = f"powershell -Command \"{author_lines_cmd}\""
    else:
        grep_pattern = '|'.join([f'^author {shlex.quote(author)}' for author in author_names])
        # Unix-like systems: using grep with extended regex
        author_lines_cmd = f"git blame --line-porcelain {relative_file_path} | grep -E '{grep_pattern}' | wc -l"

    # print(author_lines_cmd)
   
    author_lines_result = subprocess.run(author_lines_cmd, shell=True, capture_output=True, text=True)
    author_lines = int(author_lines_result.stdout.strip())
    

    return total_lines, author_lines

In [ ]:
# Test function
# fp = "c:\\school\\schweiz_UNI\\fss\\fss-se1\\se2\\elasticsearch\\core\\src\\main\\java\\org\\elasticsearch\\Build.java"
# count_lines_total_and_by_authors(fp, ["kimchy", "Shay Banon"])

In [20]:

gr = Git(repo_path)
commit = gr.get_commit_from_tag(to_tag)
gr.checkout(commit.hash)
files = defaultdict(lambda: (0 ,0))
i = 10
# change into repo path
os.chdir(repo_path)
for file in tqdm(gr.files(), desc="Analyzing files", unit="file"):
    # # lowercase first letter of filepath -> windows fix the file is returned with C not c as drive
    file = file[0].lower() + file[1:]
    total, kimchy = count_lines_total_and_by_authors(file, ['kimchy', 'Shay Banon'])
    # print(file)
    # print(total, kimchy)
    file_relative = file.replace((repo_path+"\\"), '')
    files[file_relative] = (total, kimchy)

    # i -= 1
    # if(i == 0):
    #     break
    
os.chdir("..")

Analyzing files: 100%|██████████| 6059/6059 [1:58:06<00:00,  1.17s/file]   


In [26]:
def build_json_structure(data):
    def add_to_tree(base, parts, values):
        for part in parts[:-1]:
            found = next((item for item in base if item['name'] == part), None)
            if not found:
                new_node = {"name": part, "children": []}
                base.append(new_node)
                base = new_node['children']
            else:
                base = found['children']
        
        # Add the file as a child to the last directory
        # values[0] -> total
        # values[1] -> kimchy
        to_append = {
                    "name": parts[-1], 
                    "author_color": values[0] != 0 and (values[1]/values[0] > 0.5 and "red" or "green") or "green",
                    "size": values[0], 
                    "weight": values[0],
                    "ownership":  values[0] != 0 and values[1]/values[0] or 0
                    }
        base.append(to_append)

    tree = {"name": "root", "children": []}
    
    # Sort the paths for easier processing
    sorted_data = sorted(data.items())

    for path, values in sorted_data:
        parts = path.split('\\')
        add_to_tree(tree['children'], parts, values)

    return tree


In [27]:
# Generate JSON
json_structure = build_json_structure(files)
json_output = json.dumps(json_structure, indent=2)

with open("output.json", "w") as file:
    json.dump(json_structure, file, indent=4)

print("Now run your local python server and open the knowledge_loss.html file in your browser")
print("http://localhost:8000/knowledge_loss.html")

Now run your local python server and open the knowledge_loss.html file in your browser
http://localhost:8000/knowledge_loss.html


## Task 3: Code Churn Analysis

The third and last task is to analyze the code churn of the _elasticsearch_ project. For this analysis we look at the code churn, meaning the daily change in the total number of lines of the project.

Visualize the code churn over time bucketing the data by day. Remember that you'll need to consider also the days when there are no commits.

Look at the churn trend over time, identify one outlier, and for it:

* investigate if it was caused by a single or multiple commits (since you are bucketing the data by day)
* find the hash of the involved commit(s)
* find the involved files, and for each file look at the number of lines added and/or deleted as well as the modification type (addition, deletion, modification, renaming)
* look at the commit messages

Based on the above, discuss the potential reasons for the outlier and if it should be a reason for concern.